In [1]:
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score, adjusted_rand_score


DATA_DIR = "./data"
ARTIFACTS_DIR = "./artifacts"
FIGURES_DIR = "./artifacts/figures"

os.makedirs(FIGURES_DIR, exist_ok=True)
os.makedirs(os.path.join(ARTIFACTS_DIR, "labels"), exist_ok=True)

sns.set(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

In [2]:
def calculate_metrics(X, labels):
    n_labels = len(set(labels))
    noise_ratio = np.sum(labels == -1) / len(labels) if -1 in labels else 0.0
    unique_labels_no_noise = set(labels) - {-1}
    
    if len(unique_labels_no_noise) < 2:
        return {
            "silhouette": np.nan,
            "davies_bouldin": np.nan,
            "calinski_harabasz": np.nan,
            "noise_ratio": noise_ratio,
            "n_clusters": len(unique_labels_no_noise)
        }
        
    mask = labels != -1
    X_clean = X[mask]
    labels_clean = labels[mask]
    
    return {
        "silhouette": silhouette_score(X_clean, labels_clean),
        "davies_bouldin": davies_bouldin_score(X_clean, labels_clean),
        "calinski_harabasz": calinski_harabasz_score(X_clean, labels_clean),
        "noise_ratio": noise_ratio,
        "n_clusters": len(unique_labels_no_noise)
    }

In [3]:
def plot_pca_clusters(X_processed, labels, title, filename):
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X_processed)
    
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(X_pca[:, 0], X_pca[:, 1], c=labels, cmap='viridis', s=10, alpha=0.7)
    plt.title(f"PCA Visualization: {title}")
    plt.xlabel("PC1")
    plt.ylabel("PC2")
    plt.colorbar(scatter, label="Cluster Label")
    
    save_path = os.path.join(FIGURES_DIR, filename)
    plt.savefig(save_path)
    plt.close()
    print(f"Graph saved: {save_path}")

In [4]:
def plot_metric_curve(param_values, metric_values, param_name, metric_name, title, filename):
    plt.figure(figsize=(8, 5))
    plt.plot(param_values, metric_values, marker='o')
    plt.title(title)
    plt.xlabel(param_name)
    plt.ylabel(metric_name)
    plt.grid(True)
    
    save_path = os.path.join(FIGURES_DIR, filename)
    plt.savefig(save_path)
    plt.close()

In [5]:
class ClusteringExperiment:
    def __init__(self, dataset_name, file_name):
        self.name = dataset_name
        self.file_path = os.path.join(DATA_DIR, file_name)
        self.df = None
        self.X = None
        self.X_processed = None
        self.sample_ids = None
        self.pipeline = None
        self.results = [] 
        
    def load_and_explore(self):
        print(f"\n{'='*20} Loading {self.name} {'='*20}")
        self.df = pd.read_csv(self.file_path)
        
        print("Shape:", self.df.shape)
        print("\nMissing values:\n", self.df.isna().sum()[self.df.isna().sum() > 0])
        print("\nTypes:\n", self.df.dtypes)
        display(self.df.head(3))
        
        if 'sample_id' in self.df.columns:
            self.sample_ids = self.df['sample_id']
            self.X = self.df.drop(columns=['sample_id'])
        else:
            self.sample_ids = self.df.index
            self.X = self.df
            
    def build_pipeline(self, cat_cols=None):
        num_cols = [c for c in self.X.columns if c not in (cat_cols or [])]
        
        num_transformer = Pipeline(steps=[
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ])
        
        transformers = [('num', num_transformer, num_cols)]
        
        if cat_cols:
            cat_transformer = Pipeline(steps=[
                ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
                ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
            ])
            transformers.append(('cat', cat_transformer, cat_cols))
            
        self.preprocessor = ColumnTransformer(transformers=transformers)
        
        self.X_processed = self.preprocessor.fit_transform(self.X)
        print(f"Preprocessing done. X shape: {self.X_processed.shape}")
        
    def run_kmeans(self, k_range):
        print(f"Running KMeans for k={k_range}...")
        sil_scores = []
        
        for k in k_range:
            model = KMeans(n_clusters=k, n_init=10, random_state=42)
            labels = model.fit_predict(self.X_processed)
            metrics = calculate_metrics(self.X_processed, labels)
            
            res_entry = {
                "dataset": self.name,
                "model": "KMeans",
                "params": {"n_clusters": k},
                "metrics": metrics,
                "labels": labels
            }
            self.results.append(res_entry)
            sil_scores.append(metrics['silhouette'])
            
        plot_metric_curve(k_range, sil_scores, "k", "Silhouette", 
                          f"{self.name} - KMeans Silhouette", 
                          f"{self.name}_kmeans_sil.png")

    def run_dbscan(self, eps_range, min_samples=5):
        print(f"Running DBSCAN for eps={eps_range}...")
        sil_scores = []
        
        for eps in eps_range:
            model = DBSCAN(eps=eps, min_samples=min_samples)
            labels = model.fit_predict(self.X_processed)
            metrics = calculate_metrics(self.X_processed, labels)
            
            res_entry = {
                "dataset": self.name,
                "model": "DBSCAN",
                "params": {"eps": eps, "min_samples": min_samples},
                "metrics": metrics,
                "labels": labels
            }
            self.results.append(res_entry)
            sil_scores.append(metrics['silhouette'] if not np.isnan(metrics['silhouette']) else -1)
            
        plot_metric_curve(eps_range, sil_scores, "eps", "Silhouette", 
                          f"{self.name} - DBSCAN Silhouette", 
                          f"{self.name}_dbscan_sil.png")

    def run_agglomerative(self, k_range, linkage='ward'):
        print(f"Running Agglomerative ({linkage}) for k={k_range}...")
        sil_scores = []
        for k in k_range:
            model = AgglomerativeClustering(n_clusters=k, linkage=linkage)
            labels = model.fit_predict(self.X_processed)
            metrics = calculate_metrics(self.X_processed, labels)
            
            res_entry = {
                "dataset": self.name,
                "model": f"Agglomerative_{linkage}",
                "params": {"n_clusters": k, "linkage": linkage},
                "metrics": metrics,
                "labels": labels
            }
            self.results.append(res_entry)
            sil_scores.append(metrics['silhouette'])
            
        plot_metric_curve(k_range, sil_scores, "k", "Silhouette", 
                  f"{self.name} - Agglomerative ({linkage}) Silhouette", 
                  f"{self.name}_agglo_{linkage}_sil.png")

    def save_best_solution(self, criteria='silhouette'):
        valid_results = [r for r in self.results if not np.isnan(r['metrics'][criteria])]
        
        if not valid_results:
            print("No valid results found.")
            return None

        reverse = False if criteria == 'davies_bouldin' else True
        best_res = sorted(valid_results, key=lambda x: x['metrics'][criteria], reverse=reverse)[0]
        
        print(f"\nBest solution for {self.name} based on {criteria}:")
        print(f"Model: {best_res['model']}, Params: {best_res['params']}")
        print(f"Metrics: {best_res['metrics']}")
        
        plot_pca_clusters(self.X_processed, best_res['labels'], 
                          f"{self.name} Best: {best_res['model']}", 
                          f"{self.name}_best_pca.png")
        
        labels_df = pd.DataFrame({
            'sample_id': self.sample_ids,
            'cluster_label': best_res['labels']
        })
        labels_path = os.path.join(ARTIFACTS_DIR, "labels", f"labels_{self.name}.csv")
        labels_df.to_csv(labels_path, index=False)
        print(f"Labels saved to {labels_path}")
        
        return best_res

In [6]:
DS1_FILE = "S07-hw-dataset-01.csv" 

exp1 = ClusteringExperiment("dataset_01", DS1_FILE)
exp1.load_and_explore()
exp1.build_pipeline()

exp1.run_kmeans(range(2, 11))
exp1.run_dbscan(np.arange(0.1, 1.5, 0.1), min_samples=5)

best1 = exp1.save_best_solution(criteria='silhouette')


==================== Loading dataset_01 ====================
Shape: (12000, 9)

Missing values:
 Series([], dtype: int64)

Types:
 sample_id      int64
f01          float64
f02          float64
f03          float64
f04          float64
f05          float64
f06          float64
f07          float64
f08          float64
dtype: object


,sample_id,f01,f02,f03,f04,f05,f06,f07,f08
0,0,-0.536647,-69.812900,-0.002657,71.743147,-11.396498,-12.291287,-6.836847,-0.504094
1,1,15.230731,52.727216,-1.273634,-104.123302,11.589643,34.316967,-49.468873,0.390356
2,2,18.542693,77.317150,-1.321686,-111.946636,10.254346,25.892951,44.595250,0.325893


Preprocessing done. X shape: (12000, 8)
Running KMeans for k=range(2, 11)...
Running DBSCAN for eps=[0.1 0.2 0.3 0.4 0.5 0.6 0.7 0.8 0.9 1.  1.1 1.2 1.3 1.4]...

Best solution for dataset_01 based on silhouette:
Model: KMeans, Params: {'n_clusters': 2}
Metrics: {'silhouette': 0.5216395622404242, 'davies_bouldin': 0.6853295219054459, 'calinski_harabasz': 11786.954622671528, 'noise_ratio': 0.0, 'n_clusters': 2}
Graph saved: ./artifacts/figures/dataset_01_best_pca.png
Labels saved to ./artifacts/labels/labels_dataset_01.csv


In [7]:
DS2_FILE = "S07-hw-dataset-02.csv"

exp2 = ClusteringExperiment("dataset_02", DS2_FILE)
exp2.load_and_explore()
exp2.build_pipeline()

exp2.run_kmeans(range(2, 10))

exp2.run_dbscan(np.arange(0.1, 0.8, 0.05), min_samples=5)

exp2.run_agglomerative(range(2, 10), linkage='single')

best2 = exp2.save_best_solution(criteria='silhouette')


==================== Loading dataset_02 ====================
Shape: (8000, 4)

Missing values:
 Series([], dtype: int64)

Types:
 sample_id      int64
x1           float64
x2           float64
z_noise      float64
dtype: object


,sample_id,x1,x2,z_noise
0,0,0.098849,-1.846034,21.288122
1,1,-1.024516,1.829616,6.072952
2,2,-1.094178,-0.158545,-18.938342


Preprocessing done. X shape: (8000, 3)
Running KMeans for k=range(2, 10)...
Running DBSCAN for eps=[0.1  0.15 0.2  0.25 0.3  0.35 0.4  0.45 0.5  0.55 0.6  0.65 0.7  0.75]...
Running Agglomerative (single) for k=range(2, 10)...

Best solution for dataset_02 based on silhouette:
Model: Agglomerative_single, Params: {'n_clusters': 2, 'linkage': 'single'}
Metrics: {'silhouette': 0.5213161442555723, 'davies_bouldin': 0.34186056281280663, 'calinski_harabasz': 7.184330110542804, 'noise_ratio': 0.0, 'n_clusters': 2}
Graph saved: ./artifacts/figures/dataset_02_best_pca.png
Labels saved to ./artifacts/labels/labels_dataset_02.csv


In [8]:
DS4_FILE = "S07-hw-dataset-04.csv"

exp4 = ClusteringExperiment("dataset_04", DS4_FILE)
exp4.load_and_explore()

cat_cols = exp4.X.select_dtypes(include=['object', 'category']).columns.tolist()
print("Categorical columns detected:", cat_cols)

exp4.build_pipeline(cat_cols=cat_cols)

exp4.run_kmeans(range(2, 15))
exp4.run_agglomerative(range(2, 15), linkage='ward')

best4 = exp4.save_best_solution(criteria='silhouette')


==================== Loading dataset_04 ====================
Shape: (10000, 33)

Missing values:
 n01    174
n02    189
n03    199
n04    192
n05    201
n06    183
n07    204
n08    194
n09    195
n10    189
n11    204
n12    202
n13    197
n14    198
n15    186
n16    191
n17    212
n18    212
n19    187
n20    203
n21    215
n22    196
n23    171
n24    207
n25    185
n26    224
n27    197
n28    211
n29    202
n30    195
dtype: int64

Types:
 sample_id      int64
cat_a         object
cat_b         object
n01          float64
n02          float64
n03          float64
n04          float64
n05          float64
n06          float64
n07          float64
n08          float64
n09          float64
n10          float64
n11          float64
n12          float64
n13          float64
n14          float64
n15          float64
n16          float64
n17          float64
n18          float64
n19          float64
n20          float64
n21          float64
n22          float64
n23          float64
n24

,sample_id,cat_a,cat_b,n01,n02,n03,n04,n05,n06,n07,...,n21,n22,n23,n24,n25,n26,n27,n28,n29,n30
0,0,B,X,-4.827501,-24.507466,-7.852963,0.771781,28.297884,-4.493911,-42.769449,...,24.597176,-26.354320,4.543397,-19.549036,-3.051332,-5.538587,-3.084457,5.499629,-6.128896,3.132067
1,1,F,V,51.302500,NaN,5.534737,51.305464,-8.027553,28.297548,NaN,...,-18.216260,8.527932,17.202115,-30.452260,0.855326,1.199066,3.597555,-2.239703,2.932710,0.473145
2,2,A,W,-4.820828,-2.625385,27.891578,1.523041,-5.776687,-16.298523,2.462937,...,-48.260775,9.313232,12.323411,55.081325,-3.945606,-0.280540,-0.130583,-7.353205,-2.942836,1.460477


Categorical columns detected: ['cat_a', 'cat_b']
Preprocessing done. X shape: (10000, 42)
Running KMeans for k=range(2, 15)...
Running Agglomerative (ward) for k=range(2, 15)...

Best solution for dataset_04 based on silhouette:
Model: KMeans, Params: {'n_clusters': 5}
Metrics: {'silhouette': 0.44736887827969146, 'davies_bouldin': 0.975904265483746, 'calinski_harabasz': 5087.688517434967, 'noise_ratio': 0.0, 'n_clusters': 5}
Graph saved: ./artifacts/figures/dataset_04_best_pca.png
Labels saved to ./artifacts/labels/labels_dataset_04.csv


In [9]:
print("Stability check for Dataset 01 (KMeans)")

stability_scores = []
base_model = KMeans(n_clusters=3, n_init=10, random_state=42)
base_labels = base_model.fit_predict(exp1.X_processed)

for seed in [10, 20, 30, 40, 50]:
    comp_model = KMeans(n_clusters=3, n_init=10, random_state=seed)
    comp_labels = comp_model.fit_predict(exp1.X_processed)
    
    ari = adjusted_rand_score(base_labels, comp_labels)
    stability_scores.append(ari)
    print(f"Seed {seed}: ARI = {ari:.4f}")

print(f"Average Stability (ARI): {np.mean(stability_scores):.4f}")

Stability check for Dataset 01 (KMeans)
Seed 10: ARI = 1.0000
Seed 20: ARI = 1.0000
Seed 30: ARI = 1.0000
Seed 40: ARI = 1.0000
Seed 50: ARI = 1.0000
Average Stability (ARI): 1.0000


In [10]:
all_best = [best1, best2, best4]
summary_data = {}

for item in all_best:
    if item:
        summary_data[item['dataset']] = {
            "best_model": item['model'],
            "best_params": item['params'],
            "metrics": item['metrics']
        }

with open(os.path.join(ARTIFACTS_DIR, "metrics_summary.json"), "w") as f:
    json.dump(summary_data, f, indent=4)
    